In [4]:
import xml.etree.ElementTree as ET
import json

index=4

nodes=[]
ways=[]
relations=[]

def process_element(el):
    if el.tag=="node":
        nodes.append({
            "id": int(el.attrib['id']),
            "lat": float(el.attrib['lat']),
            "lon": float(el.attrib['lon']),
            "tags": {tag.attrib["k"]: tag.attrib["v"] for tag in el}
        })
    if el.tag=="way":
        ways.append({
            "id": int(el.attrib['id']),
            "nodes": [int(nd.attrib["ref"]) for nd in el if nd.tag=="nd"],
            "tags": {tag.attrib["k"]: tag.attrib["v"] for tag in el if tag.tag=="tag"}
        }) 
    if el.tag=="relation":
        relations.append({
            "id": int(el.attrib['id']),
            "nodes": [int(nd.attrib["ref"]) for nd in el if nd.tag=="member" and nd.attrib["type"]=="node"],
            "ways": [int(nd.attrib["ref"]) for nd in el if nd.tag=="member" and nd.attrib["type"]=="way"],
            "tags": {tag.attrib["k"]: tag.attrib["v"] for tag in el if tag.tag=="tag"}
        })

# Parse the XML file
with open('ncn_japan/ncn_japan.xml', 'rb') as file:
    context = ET.iterparse(file, events=('start', 'end'))

    # Iterate over the elements
    counter = 0
    for event, element in context:
        if event == 'start':
            process_element(element)
            element.clear()
            counter += 1
            if counter % 100000 == 0:
                print(float(counter), " lines")
    

with open('ncn_japan/nodes'+'.json', 'w') as file:
    json.dump(nodes, file)
with open('ncn_japan/ways'+'.json', 'w') as file:
    json.dump(ways, file)
with open('ncn_japan/relations'+'.json', 'w') as file:
    json.dump(relations, file)